In [16]:
print('HOLA')

HOLA


In [17]:
from pathlib import Path
import numpy as np
import pandas as pd

In [18]:
#объединяем 5 параллельных измерений в одно усредненное. Получаем файл с интенсивностями
I320=np.zeros((3,72))
DATA = Path().resolve() / 'data'
for i in range(1,6):
  obj = f'({i}).csv'
  a=np.genfromtxt(DATA/ obj,delimiter=',')
  I320+=a[1:,1:]
I320/=5

In [19]:
#выводим значения туши и воды из данных
W=np.mean(I320[:,-4:],axis=1,keepdims=True)
B=np.mean(I320[:,-8:-4],axis=1,keepdims=True)
I320=I320[:,:-8]

In [20]:
#вводим матрицу концентраций
C=np.genfromtxt(DATA/'конц.txt')#в файле должны стоять точкив качестве десятичной фигни

In [21]:
#A320=LOG10((W-B)/(I-B))
A320=np.zeros_like(I320)
for i in range(3):
  A320[i,:]=np.log10((W[i,0]-B[i,0])/(I320[i,:]-B[i,0]))

In [22]:
Matrix=np.vstack([C,A320])
Matrix=np.delete(Matrix,np.where(np.logical_or(Matrix[2:]>0.9,Matrix[2:]<0.04))[1],1)
Matrix.shape 
C=Matrix[:2]
A320=Matrix[2:]
print(C)

[[6.44800e-06 1.28960e-05 1.28960e-05 2.57920e-05 2.57920e-05 5.15840e-05
  5.15840e-05 1.03168e-04 1.03168e-04 1.03168e-04 2.06336e-04 2.06336e-04
  2.06336e-04 2.06336e-04 4.12672e-04 4.12672e-04 4.12672e-04 4.12672e-04
  4.12672e-04 4.12672e-04 4.12672e-04 4.12672e-04 8.25344e-04 8.25344e-04
  8.25344e-04 8.25344e-04 8.25344e-04 8.25344e-04 8.25344e-04]
 [1.56960e-04 7.84800e-05 1.56960e-04 7.84800e-05 1.56960e-04 7.84800e-05
  1.56960e-04 3.92400e-05 7.84800e-05 1.56960e-04 1.96200e-05 3.92400e-05
  7.84800e-05 1.56960e-04 1.24260e-06 2.48520e-06 4.90500e-06 9.81000e-06
  1.96200e-05 3.92400e-05 7.84800e-05 1.56960e-04 1.24260e-06 2.48520e-06
  4.90500e-06 9.81000e-06 1.96200e-05 3.92400e-05 7.84800e-05]]


In [23]:
#делаем матрицу пересчетов по формулкам
AxA=A320@A320.T
AxA_obr=np.linalg.inv(AxA)
CxA=C@A320.T
B_m=CxA@AxA_obr

In [24]:
#считаем концентрации по расчетам
C_r=B_m@A320

In [25]:
#относительные погрешности расчетов
dC=np.abs((C-C_r)/C*100)


In [26]:
print(dC.mean())
print(dC[0,:].mean())
print(dC[1,:].mean())


68.90170322398751
39.5111262336509
98.2922802143241


In [27]:
#погрешности по разным концентрациям разных красителей
dC_FK=np.array([np.mean(dC[0,i*8:i*8+8])for i in range(8)])
dC_BS=np.array([np.mean(dC[1,i::8])for i in range(8)])
print(dC_FK)
print(dC_BS) 

[119.86389023  12.89585411   8.19079745   3.64366528          nan
          nan          nan          nan]
[ 60.34271601  15.15769286  13.9152532   16.74189998  12.17906789
  10.91030107 545.02859472 236.43763969]


c:\Users\Hipor\projects\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Hipor\projects\.venv\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
